In [ ]:
import re
import json
import os
import numpy as np
import pandas as pd
from datetime import datetime
from joblib import Parallel, delayed
import time
import requests
import logging
import sys


DEFAULT_CLASSIFICATION = {
    "type": "ДРУГОЕ",
    "product": "НЕОПРЕДЕЛЕН",
    "confidence_level": "низкий",
    "explanation": ""
}



In [ ]:
def call_gpt_oss(prompt, model="gpt-oss:120b"):
    url = ''
    api_key = ''
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Accept": "application/json",
        "Content-Type": "application/json",
    }
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": True
    }
    try:
        response = requests.post(url, json=payload, stream=True, headers=headers)
        response.raise_for_status()
        logging.info(f"response code - {response.status_code}")
        full_response = ""
        for line in response.iter_lines():
            if line:
                json_response = json.loads(line)
                token = json_response.get("response", "")
                full_response += token
                print(token, end="", flush=True)
        print()
        return full_response
    except requests.exceptions.RequestException as e:
        print(f"Error calling Ollama API: {e}")
        return None


def call_gpt_oss_embedding(prompt, model="bge-m3:latest"):
    url = ''
    api_key = ''
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Accept": "application/json",
        "Content-Type": "application/json",
    }
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": True
    }
    response = requests.post(url, json=payload, stream=False, headers=headers)
    embedding = json.loads(response.content.decode('utf-8'))['embedding']
    return embedding



In [ ]:
def clean_complaint_text(text):
    cleaned_text = re.sub(r'\d+', '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text


def load_knowledge_bases():
    with open('products.json', 'r', encoding='utf-8') as f:
        products_kb = json.load(f)
    with open('types_products.json', 'r', encoding='utf-8') as f:
        complaint_categories_kb = json.load(f)
    return products_kb, complaint_categories_kb


def create_prompt(products_kb, categories_kb, complaint_text):
    prompt_template = """
# Роль
Ты — опытный аналитик службы поддержки банка. Твоя задача — точно классифицировать клиентские жалобы на основе предоставленных справочников.

# База знаний

## Справочник продуктов:
{products_json}

## Справочник типов жалоб и продуктов:
{types_products_json}

# Инструкции по классификации:
1. Анализ текста жалобы: Внимательно прочитай жалобу и выдели ключевые слова, фразы и суть проблемы.
2. Определение Продукта: Сопоставь содержание жалобы с определениями и ключевыми словами из products.json. Выбери наиболее подходящий продукт из списка. Если жалоба затрагивает несколько продуктов, выбери основной.
3. Определение Типа: Используй types_products.json для поиска подходящего типа. Учитывай связь между типом и продуктом - тип должен быть связан с выбранным продуктом. Сопоставь ключевые слова из жалобы с type_tags и type_description.

# СТРОГИЕ ПРАВИЛА:
- Используй ТОЛЬКО названия типов и продуктов из предоставленных справочников
- Не придумывай новые названия или категории
- Если нет точного соответствия, выбери наиболее близкий вариант из существующих в справочниках
- Названия типов и продуктов должны точно соответствовать справочникам

# ФОРМАТ ОТВЕТА (JSON):
{{
    "type": "название выбранного типа",
    "product": "название выбранного продукта",
    "confidence_level": "высокий/средний/низкий",
    "explanation": "краткое обоснование выбора на основе ключевых слов из текста"
}}

Текст жалобы для классификации: 
{complaint_text}

Твой ответ:
"""
    return prompt_template.format(
        products_json=json.dumps(products_kb, ensure_ascii=False, indent=2),
        types_products_json=json.dumps(categories_kb, ensure_ascii=False, indent=2),
        complaint_text=complaint_text
    )


def parse_classification_response(response_text, error_message=""):
    if not response_text:
        classification = DEFAULT_CLASSIFICATION.copy()
        classification["explanation"] = f"Пустой ответ модели. {error_message}".strip()
        return classification

    try:
        parsed = json.loads(response_text)
        if isinstance(parsed, dict) and isinstance(parsed.get("classification"), dict):
            parsed = parsed["classification"]

        return {
            "type": parsed.get("type", DEFAULT_CLASSIFICATION["type"]),
            "product": parsed.get("product", DEFAULT_CLASSIFICATION["product"]),
            "confidence_level": parsed.get("confidence_level", DEFAULT_CLASSIFICATION["confidence_level"]),
            "explanation": parsed.get("explanation", DEFAULT_CLASSIFICATION["explanation"])
        }
    except (json.JSONDecodeError, AttributeError, TypeError) as exc:
        classification = DEFAULT_CLASSIFICATION.copy()
        classification["explanation"] = f"Ошибка парсинга: {exc}. {error_message}".strip()
        return classification



In [ ]:
def process_single_complaint(complaint_text, products_kb, categories_kb, index, text_column='text'):
    try:
        cleaned_text = clean_complaint_text(complaint_text)
        embedding = call_gpt_oss_embedding(cleaned_text, model="bge-m3:latest")
        print(embedding)

        individual_prompt = create_prompt(products_kb, categories_kb, cleaned_text)
        classification_response = call_gpt_oss(individual_prompt, model="gpt-oss:120b")
        classification_data = parse_classification_response(classification_response, error_message=f"Жалоба #{index + 1}")

        return {
            'id': index + 1,
            'original_text': complaint_text,
            'cleaned_text': cleaned_text,
            'embedding': embedding,
            'embedding_dimension': len(embedding) if embedding else 0,
            'prompt': individual_prompt,
            'classification': classification_data,
            'processed_at': datetime.now().isoformat(),
            'batch_index': index
        }

    except Exception as e:
        print(f"❌ Ошибка обработки жалобы {index}: {e}")
        fallback = DEFAULT_CLASSIFICATION.copy()
        fallback["explanation"] = f"Ошибка обработки: {str(e)}"
        return {
            'id': index + 1,
            'original_text': complaint_text,
            'cleaned_text': '',
            'embedding': None,
            'embedding_dimension': 0,
            'prompt': '',
            'classification': fallback,
            'processed_at': datetime.now().isoformat(),
            'batch_index': index
        }


def save_results(results, batch_number, total_processed, output_dir="classification_results"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{output_dir}/batch_{batch_number}_{timestamp}.json"
    print(results)

    results_data = {
        "metadata": {
            "batch_number": batch_number,
            "total_processed": total_processed,
            "timestamp": timestamp,
            "saved_at": datetime.now().isoformat(),
            "embedding_dimension": len(results[0]['embedding']) if results and results[0].get('embedding') else 0
        },
        "results": results
    }

    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(results_data, f, ensure_ascii=False, indent=2, default=str)

    print(f"✓ Batch {batch_number} сохранен: {filename} ({total_processed} жалоб)")
    return filename


def merge_all_results(output_dir="classification_results"):
    if not os.path.exists(output_dir):
        print("❌ Директория с результатами не найдена")
        return

    all_results = []
    batch_files = [f for f in os.listdir(output_dir) if f.startswith('batch_') and f.endswith('.json')]

    for batch_file in sorted(batch_files):
        batch_path = os.path.join(output_dir, batch_file)
        with open(batch_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            all_results.extend(data['results'])

    merged_file = f"{output_dir}/all_results_merged.json"
    with open(merged_file, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2, default=str)

    print(f"✅ Все результаты объединены: {merged_file}")
    print(f"📁 Всего записей: {len(all_results)}")
    return all_results


def process_complaints_parallel(complaints, batch_size=100, n_jobs=-1, output_dir="classification_results", text_column='text'):
    products_kb, categories_kb = load_knowledge_bases()
    total_complaints = len(complaints)

    print(f"🚀 Начало параллельной обработки {total_complaints} жалоб")
    print(f"🔧 Количество ядер: {n_jobs if n_jobs != -1 else 'все доступные'}")
    print(f"📦 Размер batch: {batch_size}")
    print("─" * 50)

    start_time = time.time()
    all_results = []
    batch_number = 1

    for batch_start in range(0, total_complaints, batch_size):
        batch_end = min(batch_start + batch_size, total_complaints)
        batch_complaints = complaints[batch_start:batch_end]
        batch_indices = list(range(batch_start, batch_end))

        print(f"🔄 Обработка batch {batch_number}: {batch_start}-{batch_end-1}")
        batch_results = Parallel(n_jobs=n_jobs, verbose=10)(
            delayed(process_single_complaint)(
                complaint, products_kb, categories_kb, idx
            ) for idx, complaint in zip(batch_indices, batch_complaints)
        )

        all_results.extend(batch_results)
        save_results(batch_results, batch_number, batch_end, output_dir)

        elapsed_time = time.time() - start_time
        estimated_total = (elapsed_time / batch_end) * total_complaints if batch_end else 0
        remaining_time = estimated_total - elapsed_time

        print(f"✅ Batch {batch_number} завершен: {batch_end}/{total_complaints}")
        print(f"⏱️  Прошло: {elapsed_time/60:.1f} мин | Осталось: ~{remaining_time/60:.1f} мин")
        print("─" * 50)

        batch_number += 1

    total_time = time.time() - start_time
    print(f"🎉 Обработка завершена за {total_time/60:.1f} минут")
    print(f"📊 Средняя скорость: {total_complaints/(total_time/60):.1f} жалоб/мин")

    merge_all_results(output_dir)
    return all_results


def create_comparison_dataframe(merged_results_file, original_df):
    with open(merged_results_file, 'r', encoding='utf-8') as f:
        classification_results = json.load(f)

    model_data = []
    for result in classification_results:
        classification = result.get('classification', DEFAULT_CLASSIFICATION)
        if isinstance(classification, str):
            classification = parse_classification_response(classification)
        elif not isinstance(classification, dict):
            classification = DEFAULT_CLASSIFICATION.copy()

        model_data.append({
            'index': result['id'] - 1,
            'модель_продукт': classification.get('product', '---'),
            'модель_тип_жалобы': classification.get('type', '---'),
            'модель_уверенность': classification.get('confidence_level', 'низкий'),
            'модель_обоснование': classification.get('explanation', '')
        })

    model_df = pd.DataFrame(model_data)
    model_df = model_df.set_index('index')

    comparison_df = original_df.copy()
    for col in ['модель_продукт', 'модель_тип_жалобы', 'модель_уверенность', 'модель_обоснование']:
        comparison_df[col] = model_df[col]

    return comparison_df



In [ ]:
df = pd.read_csv('complaints.csv', encoding='utf-16', sep='\t')
complaints = list(df['Описание претензии'])



In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)

results = process_complaints_parallel(
    complaints=complaints[:1],
    batch_size=100,
    n_jobs=-1,
    output_dir="parallel_results",
    text_column='text'
)

comparison_df = create_comparison_dataframe(
    merged_results_file='parallel_results/all_results_merged.json',
    original_df=df[["Описание претензии", "Тип", "Подтип", "Претензия по продукту"]]
)

# comparison_df.to_csv('final_classified_complaints.csv', index=False, encoding='utf-8-sig')
# print("✅ Итоговый датафрейм сохранен в final_classified_complaints.csv")

print("\n=== ПЕРВЫЕ 5 ЗАПИСЕЙ ===")
print(comparison_df[['модель_продукт', 'модель_тип_жалобы', 'модель_уверенность']].head())

